<a href="https://colab.research.google.com/github/andrewxu13/TextAnalytics/blob/main/C3_Text_generation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Data Acquisition!

Here we load the data into Python.


In [ ]:
f = open( "/content/a.txt" )
text = f.read()
f.close()

print(text)
print()


I like poptars and 42 and spam. Will I get spam and poptarts for the holidays? I like spam poptarts!



# Preprocessing

Here we split all the words from the sentence, and then create a dict that counts how the frequency for each word.

In [ ]:
LoW = text.split()     #LoW = List of words


d = {}

for w in LoW:

  if w not in d:
      d[w] = 1

  else:
      d[w] += 1
print(LoW)
print("There are", len(LoW), "words.")
print()
print(d)
print("There are", len(d), "distinct words.\n")

['I', 'like', 'poptars', 'and', '42', 'and', 'spam.', 'Will', 'I', 'get', 'spam', 'and', 'poptarts', 'for', 'the', 'holidays?', 'I', 'like', 'spam', 'poptarts!']
There are 20 words.

{'I': 3, 'like': 2, 'poptars': 1, 'and': 3, '42': 1, 'spam.': 1, 'Will': 1, 'get': 1, 'spam': 2, 'poptarts': 1, 'for': 1, 'the': 1, 'holidays?': 1, 'poptarts!': 1}
There are 14 distinct words.



In [ ]:
text.split()

['I',
 'like',
 'poptars',
 'and',
 '42',
 'and',
 'spam.',
 'Will',
 'I',
 'get',
 'spam',
 'and',
 'poptarts',
 'for',
 'the',
 'holidays?',
 'I',
 'like',
 'spam',
 'poptarts!']

# Feature Engineering

In here, we create a 3-vector feature for each word.

In [ ]:
"""
1) start with an empty dictionary, for each (current) word in the list, except the last one. Add the next word to the current word's list!

"""
#prepping the dictionary for the job. 1- empty dict 2- split all text 3- account for start of paragraph/line
fileDict = {}
text = text.split()
fileDict['$'] = []

for i in range(len(text)): #first, create the library
  if text[i] not in fileDict:
      #fileDict.setdefault(text[i].strip('.!?'),[]) #create a key with a default value of a list
      fileDict.setdefault(text[i].strip('.!?'),[])
for i in range(len(text)-1): #second, create the list of the library
  #word = text[i].strip('.!?')
  word = text[i]
  if i ==0:
    fileDict['$'].append(text[i])
  if word in fileDict:
    fileDict[word].append(text[i+1])
  if text[i][-1] == '.' or text[i][-1] == '?' or text[i][-1] == '!':
    fileDict['$'].append(text[i+1])

for i in list(fileDict.keys()): #delete the key with empty list
  if fileDict[i]==[]:
      del fileDict[i]

In [ ]:
fileDict

{'$': ['I', 'Will', 'I'],
 'I': ['like', 'get', 'like'],
 'like': ['poptars', 'spam'],
 'poptars': ['and'],
 'and': ['42', 'spam.', 'poptarts'],
 '42': ['and'],
 'spam': ['and', 'poptarts!'],
 'Will': ['I'],
 'get': ['spam'],
 'poptarts': ['for'],
 'for': ['the'],
 'the': ['holidays?']}

Let's also wrap this into a function so with any file, we can create the dictionary for it.

# Modeling

Based on the Markov chain, we will generate a text based on the feature engineered on fileDict.

In [ ]:
import random #use this dictionary to randomize which word to choose
results = ""
start = random.choice(list(fileDict.keys()))
results += start
next = fileDict[start]


for i in range(20): #test with generating 20 words
  word = random.choice(next)
  results = results + " " + word
  if word[-1] in '.!?':
    next = fileDict['$']
  else:
    next = fileDict[word]

In [ ]:
results #voila! here is the results

'poptars and 42 and 42 and spam. I get spam and poptarts for the holidays? I like poptars and spam. I'

# Evaluation.
Here we look at the results and do a "visual" evaluation. what do you think?

In [ ]:
results #voila! here is the results

'get spam and 42 and poptarts for the holidays? I get spam poptarts! Will I like spam poptarts! Will I like'

# Extra
Let's wrap everything in functions for reusability

In [ ]:
def createDictionary( filename ):
   """

 :param filename: string, the name of a text file containing some sample text
 :return:  should return a dictionary whose keys are words encountered in the text file and whose entries are a list of words that may legally follow the key word
   """

   file = open(filename, "r")
   text = file.read()  #read the text file and transform into text
   file.close()

#prepping the dictionary for the job. 1- empty dict 2- split all text 3- account for start of paragraph/line
   fileDict = {}
   text = text.split()
   fileDict['$'] = []

   for i in range(len(text)): #first, create the library
       if text[i] not in fileDict:
           #fileDict.setdefault(text[i].strip('.!?'),[]) #create a key with a default value of a list
           fileDict.setdefault(text[i].strip('.!?'),[])
   for i in range(len(text)-1): #second, create the list of the library
       #word = text[i].strip('.!?')
       word = text[i]
       if i ==0:
         fileDict['$'].append(text[i])
       if word in fileDict:
          fileDict[word].append(text[i+1])
       if text[i][-1] == '.' or text[i][-1] == '?' or text[i][-1] == '!':
          fileDict['$'].append(text[i+1])

   for i in list(fileDict.keys()): #delete the ley with empty list
       if fileDict[i]==[]:
           del fileDict[i]

   return fileDict

In [ ]:
def generateText(dict, nw):
    """

    :param dict: dictionary of word transitions, created by createDictionary
    :param nw: positive interger, aka number of words
    :return: string
    """
    fileDict = {}
    fileDict = createDictionary( dict )
    results = ""
    start = random.choice(list(fileDict.keys()))
    results += start
    next = fileDict[start]
    for i in range(nw):
       word = random.choice(next)
       results = results + " " + word
       if word[-1] in '.!?':
          next = fileDict['$']
       else:
          next = fileDict[word]

    return results


In [ ]:
#test it out
generateText("a.txt", 200)

'the holidays? I like poptars and spam. I get spam poptarts! Will I get spam and poptarts for the holidays? I like poptars and poptarts for the holidays? I like spam and spam. I get spam poptarts! Will I like spam poptarts! I get spam and spam. I like poptars and spam. Will I get spam poptarts! I like spam and spam. I get spam poptarts! Will I like poptars and poptarts for the holidays? I like spam poptarts! Will I like spam and 42 and poptarts for the holidays? I like poptars and poptarts for the holidays? I like spam and spam. I like poptars and poptarts for the holidays? I like spam and poptarts for the holidays? I like poptars and spam. I like spam and poptarts for the holidays? I get spam and 42 and spam. I like poptars and 42 and poptarts for the holidays? I get spam and 42 and spam. I get spam poptarts! I like poptars and poptarts for the holidays? I like poptars and poptarts for the holidays? I like spam poptarts! I like poptars and poptarts for the holidays? I like poptars and